In [60]:
import pandas as pd
from glob import  glob
import numpy as np
files=glob(r'C:\Users\v-narku\Desktop\pythonPractices\AQI-Project-master\Data\AQI\*.csv')

In [61]:
df=pd.concat((pd.read_csv(file) for file in files),ignore_index=True,axis=0)
df['PM2.5']=np.where(df['PM2.5'].isin(['NoData','PwrFail','---','InVld']),0,df['PM2.5'])
df[df['PM2.5']==0]
df['Date']=df['Date'].astype(str)
start, stop, step = 0, 10, 1
df['Date']=df['Date'].str.slice(start, stop, step)
df['Date']=pd.to_datetime(df['Date'])
df = df.set_index('Date')
df['PM2.5']= pd.to_numeric(df['PM2.5'],errors={'coerce'})
df_conso=df['PM2.5'].resample('D').mean()
df_conso=pd.DataFrame(df_conso)
df_conso.reset_index()
col=['Dates','PM2.5']
df_conso.to_csv(r'C:\Users\v-narku\Desktop\pythonPractices\Apidata.csv')

C:\Users\v-narku\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [62]:
df=pd.read_csv(r'C:\Users\v-narku\Desktop\pythonPractices\Apidata.csv',header=0)

In [63]:
import time
import requests
import sys
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv
import io
import numpy as np
data1=[]
for year in range(2013,2020):
  for month in range(1,13):
    if(month<10):
      url='http://en.tutiempo.net/climate/0{}-{}/ws-421820.html'.format(month,year)
    else:
      url='http://en.tutiempo.net/climate/{}-{}/ws-421820.html'.format(month,year)
    
    Request=requests.get(url)


    tempD = []
    finalD = []

    soup = BeautifulSoup(Request.text, "lxml")
    for table in soup.findAll('table', {'class': 'medias mensuales numspan'}):
      for tbody in table:
        for tr in tbody:
          a = tr.get_text()
          tempD.append(a)

    rows = len(tempD) / 15

    for times in range(round(rows)):
      newtempD = []
      for i in range(15):
        newtempD.append(tempD[0])
        tempD.pop(0)
        finalD.append(newtempD)
    data=pd.DataFrame(finalD,columns=['Day', 'T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM', 'VG', 'RA', 'SN', 'TS', 'FG'])
    data.drop_duplicates(keep='first',inplace=True)
    data.drop(data.head(1).index,axis=0,inplace=True)
    data.drop(data.tail(2).index,axis=0,inplace=True)
    data['dates']=pd.to_datetime(str(year)+'-'+str(month)+'-'+ data['Day'])
    data.drop(columns=['VG', 'RA', 'SN', 'TS', 'FG','Day','PP'],axis=1,inplace=True)
    data['T']=np.where(data['T']=='',0,data['T'])
    data.drop(data[data['T']==0].index,axis=0,inplace=True)
    data1=pd.DataFrame(data1) 
    data1=pd.concat([data,data1],ignore_index=True)

In [64]:
# df=pd.read_csv(r'C:\Users\v-narku\Desktop\pythonPractices\combine.csv')
apidata=pd.read_csv(r'C:\Users\v-narku\Desktop\pythonPractices\Apidata.csv',header=0)
data1['dates']=pd.to_datetime(data1['dates'])
data1['year']=data1['dates'].dt.year.astype(str)
apidata['Date']=pd.to_datetime(apidata['Date'])
df3=data1[data1['year'].isin(['2013','2014','2015'])]
df2=df3.merge(apidata,left_on='dates',right_on='Date',how='left')
df2.drop(columns=['Date'],inplace=True)
df2['VM']=pd.to_numeric(df2['VM'],errors='coerce')
df2.drop(columns=['dates','year'],inplace=True)
df2.to_csv(r'C:\Users\v-narku\Desktop\pythonPractices\Real_data.csv',index=False)

# EDA